## <b>Step1:-importing necessary libraries

In [62]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import re
import nltk
import string
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer,WordNetLemmatizer
from sklearn.feature_extraction.text import CountVectorizer
import xgboost as xgb
from sklearn.metrics import accuracy_score,precision_score,recall_score,roc_auc_score,classification_report


## <b>Step2:-Data Ingestion

In [8]:
df=pd.read_csv("https://raw.githubusercontent.com/entbappy/Branching-tutorial/refs/heads/master/tweet_emotions.csv")
df.to_csv("tweet_emotions.csv")

In [9]:
df.head()

,tweet_id,sentiment,content
0,1956967341,empty,@tiffanylue i know i was listenin to bad habi...
1,1956967666,sadness,Layin n bed with a headache ughhhh...waitin o...
2,1956967696,sadness,Funeral ceremony...gloomy friday...
3,1956967789,enthusiasm,wants to hang out with friends SOON!
4,1956968416,neutral,@dannycastillo We want to trade with someone w...


In [22]:
df.shape

(40000, 2)

In [11]:
df.drop(columns=['tweet_id'],inplace=True)
df.head()

,sentiment,content
0,empty,@tiffanylue i know i was listenin to bad habi...
1,sadness,Layin n bed with a headache ughhhh...waitin o...
2,sadness,Funeral ceremony...gloomy friday...
3,enthusiasm,wants to hang out with friends SOON!
4,neutral,@dannycastillo We want to trade with someone w...


In [13]:
df['sentiment'].unique()

array(['empty', 'sadness', 'enthusiasm', 'neutral', 'worry', 'surprise',
       'love', 'fun', 'hate', 'happiness', 'boredom', 'relief', 'anger'],
      dtype=object)

In [14]:
df['sentiment'].isin(['happiness','sadness'])

0        False
1         True
2         True
3        False
4        False
         ...  
39995    False
39996    False
39997    False
39998     True
39999    False
Name: sentiment, Length: 40000, dtype: bool

In [17]:
final_df=df[df['sentiment'].isin(['happiness','sadness'])]
final_df.head()

,sentiment,content
1,sadness,Layin n bed with a headache ughhhh...waitin o...
2,sadness,Funeral ceremony...gloomy friday...
6,sadness,"I should be sleep, but im not! thinking about ..."
8,sadness,@charviray Charlene my love. I miss you
9,sadness,@kelcouch I'm sorry at least it's Friday?


In [19]:
final_df['sentiment'].unique()

array(['sadness', 'happiness'], dtype=object)

In [23]:
final_df.shape

(10374, 2)

In [24]:
final_df['sentiment'].replace({'happiness':1,'sadness':0},inplace=True)

C:\Users\pande\AppData\Local\Temp\ipykernel_5400\3433959475.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  final_df['sentiment'].replace({'happiness':1,'sadness':0},inplace=True)
C:\Users\pande\AppData\Local\Temp\ipykernel_5400\3433959475.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_df['sentiment'].replace({'happiness':1,'sadness':0},inplace=True)


In [26]:
final_df.head()

,sentiment,content
1,0,Layin n bed with a headache ughhhh...waitin o...
2,0,Funeral ceremony...gloomy friday...
6,0,"I should be sleep, but im not! thinking about ..."
8,0,@charviray Charlene my love. I miss you
9,0,@kelcouch I'm sorry at least it's Friday?


In [28]:
final_df['sentiment'].unique()

array([0, 1])

In [27]:
final_df.shape

(10374, 2)

In [29]:
train_data,test_data=train_test_split(final_df,test_size=0.2,random_state=4)

In [31]:
train_data.head()

,sentiment,content
25824,0,@shonali hello- i'd like a little chat- but ca...
38684,0,Went shopping for sports tour yesterday with G...
10564,1,@Minicooperr yeahh! Ryland was amazing xD BUT ...
9525,0,the dog dug a hole in the backyard well iwas s...
24287,1,"Went to see Priscilla Ahn last night, was amaz..."


In [32]:
train_data.shape

(8299, 2)

In [33]:
test_data.head()

,sentiment,content
37839,1,"@GeekySteph also, try Friendly Fires if you ha..."
5442,0,i need to get my life together aka my apartmen...
21094,1,@souravghosh Dreams give rise to reality. When...
28755,1,@lilcdawg its because we run on CPT!!!
6382,0,The computer at work hates me. have to call t...


In [34]:
test_data.shape


(2075, 2)

## <b>Step3:-Data Preprocessing

In [ ]:
# nltk.download('wordnet')
# nltk.download('stopwords')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\pande\AppData\Roaming\nltk_data...
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\pande\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


True

In [ ]:
# def lemmatization(text):
#     lemmatizer=WordNetLemmatizer()
#     text=text.split()
#     text=[lemmatizer.lemmatize(y) for y in text]
#     return " ".join(text)

In [ ]:
# def remove_stop_words(text):
#     stop_words=set(stopwords.words("english"))
#     Text=[i for i in str(text).split() if i not in stop_words]
#     return " ".join(text)

In [ ]:
# def removing_numbers(text):
#     text=''.join([i for i in text if not i.isdigit()])
#     return text

In [ ]:
# def lower_case(text):
#     text=text.split()
#     text=[y.lower() for y in text]
#     return " ".join(text)

In [ ]:
# import re

# def removing_punctuations(text):
#     # Define punctuations as a raw string
#     punctuations = r"""!"#$%&'()*+,،-./:;<=>؟?@[\]^_`{|}~"""

#     # Remove punctuations
#     text = re.sub(r'[%s]' % re.escape(punctuations), ' ', text)
#     text = text.replace('؛', "")

#     # Remove extra whitespace
#     text = re.sub(r'\s+', ' ', text)
#     text = " ".join(text.split())

#     return text.strip()


In [ ]:
# def removing_urls(text):
#     url_pattern = re.compile(r'https?://\S+|www\.\S+')
#     return url_pattern.sub(r'', text)

In [ ]:
# def normalize_text(df):
#     df.content=df.content.apply(lambda content : lower_case(content))
#     df.content=df.content.apply(lambda content : remove_stop_words(content))
#     df.content=df.content.apply(lambda content : removing_numbers(content))
#     df.content=df.content.apply(lambda content : removing_punctuations(content))
#     df.content=df.content.apply(lambda content : removing_urls(content))
#     df.content=df.content.apply(lambda content : lemmatization(content))
#     return df

In [ ]:
# def normalized_sentence(sentence):
#     sentence= lower_case(sentence)
#     sentence= remove_stop_words(sentence)
#     sentence= removing_numbers(sentence)
#     sentence= removing_punctuations(sentence)
#     sentence= removing_urls(sentence)
#     sentence= lemmatization(sentence)
#     return sentence

In [51]:
nltk.download('wordnet')
nltk.download('stopwords')

def lemmatization(text):
    lemmatizer= WordNetLemmatizer()

    text = text.split()

    text=[lemmatizer.lemmatize(y) for y in text]

    return " " .join(text)

def remove_stop_words(text):
    stop_words = set(stopwords.words("english"))
    Text=[i for i in str(text).split() if i not in stop_words]
    return " ".join(Text)

def removing_numbers(text):
    text=''.join([i for i in text if not i.isdigit()])
    return text

def lower_case(text):

    text = text.split()

    text=[y.lower() for y in text]

    return " " .join(text)

def removing_punctuations(text):
    ## Remove punctuations
    text = re.sub('[%s]' % re.escape("""!"#$%&'()*+,،-./:;<=>؟?@[\]^_`{|}~"""), ' ', text)
    text = text.replace('؛',"", )

    ## remove extra whitespace
    text = re.sub('\s+', ' ', text)
    text =  " ".join(text.split())
    return text.strip()

def removing_urls(text):
    url_pattern = re.compile(r'https?://\S+|www\.\S+')
    return url_pattern.sub(r'', text)

def remove_small_sentences(df):
    for i in range(len(df)):
        if len(df.text.iloc[i].split()) < 3:
            df.text.iloc[i] = np.nan

def normalize_text(df):
    df.content=df.content.apply(lambda content : lower_case(content))
    df.content=df.content.apply(lambda content : remove_stop_words(content))
    df.content=df.content.apply(lambda content : removing_numbers(content))
    df.content=df.content.apply(lambda content : removing_punctuations(content))
    df.content=df.content.apply(lambda content : removing_urls(content))
    df.content=df.content.apply(lambda content : lemmatization(content))
    return df

def normalized_sentence(sentence):
    sentence= lower_case(sentence)
    sentence= remove_stop_words(sentence)
    sentence= removing_numbers(sentence)
    sentence= removing_punctuations(sentence)
    sentence= removing_urls(sentence)
    sentence= lemmatization(sentence)
    return sentence

<>:32: SyntaxWarning: invalid escape sequence '\]'
<>:36: SyntaxWarning: invalid escape sequence '\s'
<>:32: SyntaxWarning: invalid escape sequence '\]'
<>:36: SyntaxWarning: invalid escape sequence '\s'
C:\Users\pande\AppData\Local\Temp\ipykernel_5400\2711691001.py:32: SyntaxWarning: invalid escape sequence '\]'
  text = re.sub('[%s]' % re.escape("""!"#$%&'()*+,،-./:;<=>؟?@[\]^_`{|}~"""), ' ', text)
C:\Users\pande\AppData\Local\Temp\ipykernel_5400\2711691001.py:36: SyntaxWarning: invalid escape sequence '\s'
  text = re.sub('\s+', ' ', text)
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\pande\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\pande\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [52]:
normalized_sentence("That's it? It's done already? This is one")

'that s it done already one'

In [53]:
train_data = normalize_text(train_data)
test_data = normalize_text(test_data)

In [54]:
train_data

,sentiment,content
25824,0,shonali hello like little chat can t dm you ma...
38684,0,went shopping sport tour yesterday garley funn...
10564,1,minicooperr yeahh ryland amazing xd get hug bo...
9525,0,dog dug hole backyard well iwas sleeping im gr...
24287,1,went see priscilla ahn last night amazin band ...
...,...,...
1693,0,heavily medicated can t sleep ugh
22789,1,land pillow blanket mm fan high mention blanke...
2673,1,hilsatlvsonline thanks tip unfortunately major...
32155,1,hungraaaaaaaaay eat go away twitter whatchandr...


In [55]:
test_data

,sentiment,content
37839,1,geekysteph also try friendly fire already hear...
5442,0,need get life together aka apartment cleaned b...
21094,1,souravghosh dream give rise reality said walk ...
28755,1,lilcdawg run cpt
6382,0,computer work hate me call tech support again
...,...,...
14367,0,sad cant go friend house
37238,1,photo thelovelybones spy jonas brother demi lo...
5364,0,nettagyrl make sad follow someone converse sim...
14741,1,back home bathing kid ironing clothes hubby ne...


## <b>Step4:-Feature Engineering

In [56]:
X_train = train_data['content'].values
y_train = train_data['sentiment'].values

X_test = test_data['content'].values
y_test = test_data['sentiment'].values

In [57]:
# Apply Bag of Words (CountVectorizer)
vectorizer = CountVectorizer()

# Fit the vectorizer on the training data and transform it
X_train_bow = vectorizer.fit_transform(X_train)

# Transform the test data using the same vectorizer
X_test_bow = vectorizer.transform(X_test)

In [58]:
train_df = pd.DataFrame(X_train_bow.toarray())

train_df['label'] = y_train
train_df.head()

,0,1,2,3,4,5,6,7,8,9,...,14212,14213,14214,14215,14216,14217,14218,14219,14220,label
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


## <b>Step5:-Model Building

In [64]:
# Define and train the XGBoost model
xgb_model = xgb.XGBClassifier(use_label_encoder=False, eval_metric='mlogloss')
xgb_model.fit(X_train_bow, y_train)

# Make predictions
y_pred = xgb_model.predict(X_test_bow)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
classification_rep = classification_report(y_test, y_pred)

print("Accuracy:", accuracy)
print("Classification Report:\n", classification_rep)

c:\Users\pande\miniconda3\envs\env_for_GenAi\Lib\site-packages\xgboost\training.py:183: UserWarning: [10:20:07] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


Accuracy: 0.776867469879518
Classification Report:
               precision    recall  f1-score   support

           0       0.75      0.84      0.79      1041
           1       0.81      0.72      0.76      1034

    accuracy                           0.78      2075
   macro avg       0.78      0.78      0.78      2075
weighted avg       0.78      0.78      0.78      2075



## <b>Step6:-Model Evaluation

In [65]:
# Make predictions
y_pred = xgb_model.predict(X_test_bow)
y_pred_proba = xgb_model.predict_proba(X_test_bow)[:, 1]

# Calculate evaluation metrics
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
auc = roc_auc_score(y_test, y_pred_proba)

In [66]:
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"AUC: {auc}")

Precision: 0.8140814081408141
Recall: 0.7156673114119922
AUC: 0.8627514646124002
